In [1]:
import torch
from torch.distributions import constraints
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
pyro.enable_validation(True)    # <---- This is always a good idea!

In [2]:
data = torch.zeros(10)
data[0:6] = 1.0

def train(model, guide, lr=0.01):
    pyro.clear_param_store()
    adam = pyro.optim.Adam({"lr": lr})
    svi = SVI(model, guide, adam, loss=Trace_ELBO())

    n_steps = 101
    for step in range(n_steps):
        loss = svi.step(data)
        if step % 50 == 0:
            print('[iter {}]  loss: {:.4f}'.format(step, loss))

## ベルヌーイ分布のベイジアンモデル

In [18]:
def model(data):
    f = pyro.sample("latent_fairness", dist.Beta(10.0, 10.0))
    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

In [19]:
def guide_bayesian(data):
    # register the two variational parameters with Pyro.
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

In [20]:
train(model, guide_bayesian)

[iter 0]  loss: 6.6669
[iter 50]  loss: 7.0721
[iter 100]  loss: 7.4315


## ベルヌーイ分布の最尤推定モデル
 
 潜在変数を分布ではなく pyro.paramを用いて、実数にする

In [6]:
def model_mle(data):
    # unit_intervalは(0, 1)の範囲指定
    f = pyro.param("latent_fairness", torch.tensor(0.5),
                   constraint=constraints.unit_interval) 
    with pyro.plate("data", data.size(0)):
        pyro.sample("obs", dist.Bernoulli(f), obs=data)

In [7]:
def guide_mle(data):
    pass

In [8]:
train(model_mle, guide_mle)

[iter 0]  loss: 6.9315
[iter 50]  loss: 6.7310
[iter 100]  loss: 6.7301


In [9]:
print("Our MLE estimate of the latent fairness is {:.3f}".format(
      pyro.param("latent_fairness").item()))

Our MLE estimate of the latent fairness is 0.601


## ベルヌーイ分布のMAP推定

事後分布における潜在変数を退化分布として仮定する.
このとき、`model`にて定義した事前分布、つまりは`latent_fairness`の分布${\rm Beta(10, 10)}の平均は5になるため、最尤推定に比べて上に

In [11]:
def guide_map(data):
    f_map = pyro.param("f_map", torch.tensor(0.5),
                       constraint=constraints.unit_interval)
    # 唯一の値を持つ確率分布: 退化分布
    pyro.sample("latent_fairness", dist.Delta(f_map))

In [15]:
train(model, guide_map)

[iter 0]  loss: 5.6719
[iter 50]  loss: 5.6006
[iter 100]  loss: 5.6004


In [16]:
print("Our MAP estimate of the latent fairness is {:.3f}".format(
      pyro.param("f_map").item()))

Our MAP estimate of the latent fairness is 0.536
